#### Time(t), Voltage(V), Load Torque(LT), Current(I), Speed(w)
Control variables - V, LT  
State variables - I, w

### Read data

In [2]:
t_file = []
voltage = []
loadTorque = []
current = []
speed = []


f = open('DC_motor_001.txt')
for line in f:
    values = line.split()
    t_file.append(float(values[0]))
    voltage.append(float(values[1]))
    loadTorque.append(float(values[2]))
    current.append(float(values[3]))
    speed.append(float(values[4]))
    

#### Test print

In [ ]:
t_file

### Fit sparse and non-sparse model

In [25]:
import numpy as np
import pysindy as ps

optNonSparse = ps.STLSQ(threshold=0, fit_intercept=False)
optSparse = ps.STLSQ(threshold=10.0, fit_intercept=False)

X = np.column_stack((current, speed))
U = np.column_stack((voltage, loadTorque))
t = np.array(t_file)
featureNames = ['i', 'w', 'V', 'LT']


varsLib = ps.PolynomialLibrary(degree=2, include_interaction=False)
controlLib = ps.PolynomialLibrary(degree=1, include_interaction=False)

customVarsLib = ps.CustomLibrary(
    library_functions=[lambda x: x, lambda x: x**2, lambda x: x**3],
    function_names=[lambda x: x, lambda x: x + '^2', lambda x: x + '^3']
    )
customControlLib = ps.CustomLibrary(
    library_functions=[lambda x: x],
    function_names=[lambda x: x]
    )

inputs_temp = [[0,1,1,1],
                [2,3,3,3]]
inputs_per_library = np.asarray(inputs_temp)

featureLibrary = ps.GeneralizedLibrary(
    [customVarsLib, customControlLib],
    inputs_per_library = inputs_per_library
)

modelNonSparse = ps.SINDy(feature_names=featureNames, optimizer=optNonSparse,feature_library=featureLibrary)
modelNonSparse.fit(X, t=t, u=U)

modelSparse = ps.SINDy(feature_names=featureNames, optimizer=optSparse, feature_library=featureLibrary)
modelSparse.fit(X, t=t, u=U)


c:\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.56031e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
c:\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.56031e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
c:\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.56031e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
c:\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.56031e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


SINDy(differentiation_method=FiniteDifference(),
      feature_library=<pysindy.feature_library.generalized_library.GeneralizedLibrary object at 0x0000019775289610>,
      feature_names=['i', 'w', 'V', 'LT'], optimizer=STLSQ(threshold=10.0))

### Sparse Model

In [26]:
modelSparse.print()

(i)' = -309.390 i + -31.007 w + 619.466 V
(w)' = 199.030 i + -4103.977 LT


### Non-Sparse Model

In [27]:
modelNonSparse.print()

(i)' = -309.702 i + -30.963 w + 0.001 i^2 + 618.837 V + 2.791 LT
(w)' = 204.978 i + 0.120 w + -10.442 V + -3987.292 LT
